# Project 2c

In [1]:
from __future__ import print_function
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pandas
import collections
from math import lgamma

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
largeDf = pandas.read_csv('large2.csv')

In [3]:
states = sorted(largeDf.s.unique())#range(0,10101010)
actions = sorted(largeDf.a.unique())
stateprimes = largeDf.sp.unique()

In [4]:
rewards = largeDf.r.unique()

In [33]:
len(states)

861064

In [18]:
'%08d' % 9763087

'09763087'

In [5]:
rewards

array([  0,  -5,  10,   3, -10, -20,   5,  13,  -2, -15,  -7,  20,   6,
       -25,  -4,  16,   1, -12,   9,   8,  30,  23])

In [19]:
larger_rewards = largeDf[largeDf['r']>0].groupby(['s', 'a', 'r']).size().reset_index(name='counts').sort_values(by=['s'])

In [87]:
punishments = largeDf[largeDf['r']<0].groupby(['s', 'a', 'r']).size().reset_index(name='counts').sort_values(by=['s'])

In [89]:
not_zero_reward = largeDf[largeDf['r']!=0].groupby(['s', 'a', 'r']).size().reset_index(name='counts').sort_values(by=['s'])

In [88]:
len(punishments)

46287

In [86]:
len(larger_rewards[larger_rewards['counts']>1])

45

In [61]:
ra = {}

for row in larger_rewards.itertuples():
    if row.s not in ra:
        ra[row.s] = {}
    if row.a not in ra[row.s]:
        ra[row.s][row.a] = []
    ra[row.s][row.a].append(row.r)

In [62]:
ra_unique = {}

for row in larger_rewards.itertuples():
    if row.s not in ra_unique:
        ra_unique[row.s] = {}
    if row.a not in ra_unique[row.s]:
        ra_unique[row.s][row.a] = []
    for a in ra[row.s].keys():
        ra_unique[row.s][a] = np.unique(ra[row.s][a])[0]

In [63]:
len(ra_unique)

44602

In [80]:
for k,v in ra_unique.items():
    for k1,v1 in v.items():
        if len(v1) > 1:
            print(k,k1)

2114887 114
2567396 22


In [84]:
reward_map = {}
for s,a_map in ra_unique.items():
    reward_map[s] = {}
    for a,r_list in a_map.items():
        reward_map[s][a] = r_list[0]

In [85]:
reward_map[2114887]

{114: 3}

In [83]:
ra_unique[2114887]

{114: array([ 3, 10])}

In [67]:
ra = {}
for s in range(0,8):
    ra[s] = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{}}

for row in larger_rewards.itertuples():
    for index,char in enumerate('%08d' % row.s):
        if row.a not in ra[index][int(char)]:
            ra[index][int(char)][row.a] = []
        ra[index][int(char)][row.a].append(row.r)

In [71]:
r_unique = {}
for s in range(0,8):
    r_unique[s] = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{}}

for s in range(0,8):
    for s1 in range(0,10):
        r_unique[s][s1] = {}
        for a in ra[s][s1].keys():
            r_unique[s][s1][a] = np.unique(ra[s][s1][a])

In [96]:
ra = {}

for row in not_zero_reward.itertuples():
    if row.s not in ra:
        ra[row.s] = {}
    if row.a not in ra[row.s]:
        ra[row.s][row.a] = []
    ra[row.s][row.a].append(row.r)
#     if row.r < 0:
#         print(row.s,row.a)
#         break

In [98]:
ra[1347]

{32: [-10]}

In [5]:
not_zero_reward = largeDf[largeDf['r']!=0].groupby(['s', 'a', 'r']).size().reset_index(name='counts').sort_values(by=['s'])

In [6]:
def create_reward_map(df):
    ra = {}

    for row in df.itertuples():
        if row.s not in ra:
            ra[row.s] = {}
        if row.a not in ra[row.s]:
            ra[row.s][row.a] = []
        ra[row.s][row.a].append(row.r)
    
    ra_unique = {}

    for row in df.itertuples():
        if row.s not in ra_unique:
            ra_unique[row.s] = {}
        if row.a not in ra_unique[row.s]:
            ra_unique[row.s][row.a] = []
        for a in ra[row.s].keys():
            l = np.unique(ra[row.s][a])
            if len(l)>1:
                print(row.s,a,l)
            ra_unique[row.s][a] = l[0]

    return ra_unique

In [7]:
reward_map = create_reward_map(not_zero_reward)

2114887 114 [ 3 10]
2114887 114 [ 3 10]
2567396 22 [10 20]
2567396 22 [10 20]
4450557 63 [-10  -5]
4450557 63 [-10  -5]
5354147 96 [-15  -5]
5354147 96 [-15  -5]
7113537 44 [-10  -5]
7113537 44 [-10  -5]


In [8]:
states = range(0,10101010)

In [9]:
def get_reward(reward_map, state, action):
    if state in reward_map and action in reward_map[state]:
        return reward_map[state][action]
    else:
        return 0

In [10]:
def calc_T(df, states, actions):
    T = {}
    for s in states:
        T[s] = {}

    for row in df.itertuples():
        if row.a not in T[row.s]:
            T[row.s][row.a] = {}
        T[row.s][row.a][row.sp] = row.counts
    return T

In [11]:
def calc_T_prob(T, states, actions):
    T_prob = {}
    for s in states:
        T_prob[s] = {}
        for a in actions:
            T_prob[s][a] = collections.defaultdict(lambda: 0)
            if a not in T[s]:
                continue
#             if a not in T[s]:
#                 for n in [1,-1,2,-2,3,-3,4,-4]:
#                     if (s+n) in T and a in T[s+n]:
#                         T[s][a] = T[s+n][a]
#                 if a not in T[s]:
#                     print("No neighbors of ", s," had ", a)
#                     continue

            sum_val = sum(T[s][a].values())
            poss_states = T[s][a].keys()
            for sp in poss_states:
                T_prob[s][a][sp] = T[s][a][sp]/sum_val
    return T_prob

In [ ]:
sasp_counts = largeDf.groupby(['s', 'a', 'sp']).size().reset_index(name='counts')
T = calc_T(sasp_counts, states, actions)
T_prob = calc_T_prob(T, states, actions)

In [39]:
pos_reward_df = mediumDf[mediumDf['r'] > 0].groupby(['r', 'sp']).size().reset_index(name='counts')

In [103]:
discount = 0.95
    
def policy_iter(states, actions, T_prob, iters=10):
    policy = {}
    U = {}
    for s in states:
        U[s] = 0
    for i in range(iters):
        for s in states:
            action_results = []
            for a in actions:
                sp_util = []
                for sp in T_prob[s][a].keys():
                    if sp in T_prob[s][a] and sp in U:
                        sp_util.append(T_prob[s][a][sp]*U[sp])
                action_results.append(get_reward(s,a) + discount*sum(sp_util))
            U[s] = max(action_results)
            policy[s] = np.argmax(action_results)+1 ## actions are 1-based
    return policy, U

In [104]:
policy, utility = policy_iter(states, actions, T_prob, 1)

In [105]:
len(policy.keys())

50000

In [106]:
len(utility.keys())

50000

In [107]:
len(states)

50000

In [117]:
def save_policy(policy, filename):
    policy_sorted = []
    for s in states:
        policy_sorted.append(str(policy[s]))
    policy_string = '\n'.join(policy_sorted)
    with open(filename, 'w') as f:
        f.write(policy_string)
        f.write('\n4')

In [118]:
save_policy(policy, 'medium.policy')